[link](https://docs.google.com/document/d/1h_1_UxUdzhkjzhMZCcibNOufkU9asa7cdWIZHOFeUMc/edit#heading=h.6qxxdaq5ec9c)


# Setup





In [82]:
# Manipulação de dados
import pandas as pd
import numpy as np

Olá candidato(a), o objetivo deste desafio é testar os seus conhecimentos sobre a resolução de problemas de negócios, análise de dados e aplicação de modelos preditivos. Queremos testar seus conhecimentos dos conceitos estatísticos de modelos preditivos, criatividade na resolução de problemas e aplicação de modelos básicos de machine learning.  É importante deixar claro que não existe resposta certa e que o que nos interessa é sua capacidade de descrever e justificar os passos utilizados na resolução do problema. 

## Desafio
Você foi alocado em um time da Indicium contratado por um estúdio de Hollywood chamado PProductions, e agora deve fazer uma análise em cima de um banco de dados cinematográfico para orientar qual tipo de filme deve ser o próximo a ser desenvolvido. Lembre-se que há muito dinheiro envolvido, então a análise deve ser muito detalhada e levar em consideração o máximo de fatores possíveis (a introdução de dados externos é permitida - e encorajada).

## Entregas

- Faça uma análise exploratória dos dados (EDA), demonstrando as principais características entre as variáveis e apresentando algumas hipóteses relacionadas. Seja criativo!
- Responda também às seguintes perguntas:
- - Qual filme você recomendaria para uma pessoa que você não conhece?
- - Quais são os principais fatores que estão relacionados com alta expectativa de faturamento de um filme? 
- - Quais insights podem ser tirados com a coluna Overview? É possível inferir o gênero do filme a partir dessa coluna?
- Explique como você faria a previsão da nota do imdb a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?
- Supondo um filme com as seguintes características:

```
{'Series_Title': 'The Shawshank Redemption',
 'Released_Year': '1994',
 'Certificate': 'A',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Meta_score': 80.0,
 'Director': 'Frank Darabont',
 'Star1': 'Tim Robbins',
 'Star2': 'Morgan Freeman',
 'Star3': 'Bob Gunton',
 'Star4': 'William Sadler',
 'No_of_Votes': 2343110,
 'Gross': '28,341,469'}
 ```

- Qual seria a nota do IMDB?

- Salve o modelo desenvolvido no formato .pkl. 

**A entrega deve ser feita através de um repositório de código público que contenha:**
- README explicando como instalar e executar o projeto
- Arquivo de requisitos com todos os pacotes utilizados e suas versões
- Relatórios das análises estatísticas e EDA em PDF, Jupyter Notebook ou semelhante conforme passo 1 e 2.
- Códigos de modelagem utilizados no passo 3 (pode ser entregue no mesmo Jupyter Notebook).
- Arquivo .pkl conforme passo 5 acima.
- Todos os códigos produzidos devem seguir as boas práticas de codificação.



# Metodologia CRISP-DM

Este projeto será estruturado seguindo a metodologia CRISP-DM (Cross Industry Standard Process for Data Mining). 

## Fases da Metodologia CRISP-DM
![CRISP DM](Imagens/wi-crisp-en2.jpg)




1. **Entendimento do Negócio**
   - Compreender os objetivos e requisitos do projeto do ponto de vista do negócio.
   - Traduzir essas metas em uma definição do problema que possa ser abordada com dados.

2. **Entendimento dos Dados**
   - Coletar dados iniciais e familiarizar-se com eles.
   - Identificar problemas de qualidade dos dados e começar a encontrar insights iniciais.

3. **Preparação dos Dados**
   - Selecionar e limpar os dados relevantes para a análise.
   - Transformar e formatar os dados conforme necessário para as etapas subsequentes.

4. **Modelagem**
   - Selecionar e aplicar técnicas de modelagem apropriadas.
   - Calibrar parâmetros do modelo para otimizar o desempenho.

5. **Avaliação**
   - Avaliar os modelos para garantir que eles atendam aos objetivos do negócio.
   - Revisar todas as etapas anteriores e verificar se há alguma questão que ainda precisa ser abordada.

6. **Implementação**
   - Implementar o modelo em um ambiente de produção.
   - Documentar o processo e preparar um relatório final com os resultados e conclusões.

## Referência

Para mais informações sobre a metodologia CRISP-DM, consulte a documentação original:

- CRISP-DM 1.0: Step-by-step data mining guide. SPSS Inc. 
[Link para o documento](https://www.kde.cs.uni-kassel.de/wp-content/uploads/lehre/ws2012-13/kdd/files/CRISPWP-0800.pdf)



# 1 - Entendimento do Negócio

Inicialmente podemos definir um objetivo principal que é: **identificar qual tipo de filme a PProductions deve desenvolver a seguir para maximizar o retorno sobre o investimento.**

Para isso podemos classificar algumas metas intermediárias para se aprofundar no entendimento do negócio.
 
- Determinar os gêneros de filmes mais lucrativos.
- Analisar tendências de lançamento bem-sucedidas ao longo dos anos.
- Identificar fatores de sucesso (elenco, diretor, enredo).

Também é importante buscar métricas para quantificar.



# 2 - Entendimento dos Dados

###  Dicionário de dados.

A base de dados de treinamento contém 15 colunas. Seus nomes são auto-explicativos, mas, caso haja alguma dúvida, a descrição das colunas é:

| Series_Title | Nome do filme                          |
|--------------|----------------------------------------|
| Released_Year| Ano de lançamento                      |
| Certificate  | Classificação etária                   |
| Runtime      | Tempo de duração                       |
| Genre        | Gênero                                 |
| IMDB_Rating  | Nota do IMDB                           |
| Overview     | Overview do filme                      |
| Meta_score   | Média ponderada de todas as críticas   |
| Director     | Diretor                                |
| Star1        | Ator/atriz #1                          |
| Star2        | Ator/atriz #2                          |
| Star3        | Ator/atriz #3                          |
| Star4        | Ator/atriz #4                          |
| No_of_Votes  | Número de votos                        |
| Gross        | Faturamento                            |



In [83]:
# Carregamento inicial dos dados
data_path = 'Data/desafio_indicium_imdb.csv'
df = pd.read_csv(data_path)\
       .drop('Unnamed: 0', axis = 1)

In [84]:
# Leitura das 5 primeiras linhas dos dados
df.head()


,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


* É possivel notar que na coluna `Genre` possui multiplos generos separados por vírgula.
* Os atores podem se repetir em colunas diferentes.
* A coluna `Runtime` referente ao tempo de filme necessita de correção para a remoção dos caracteres indesejados e transformação para um tipo de dado númerico.
* A coluna `Certificate` pode ser tratada como categorica.

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   999 non-null    object 
 1   Released_Year  999 non-null    object 
 2   Certificate    898 non-null    object 
 3   Runtime        999 non-null    object 
 4   Genre          999 non-null    object 
 5   IMDB_Rating    999 non-null    float64
 6   Overview       999 non-null    object 
 7   Meta_score     842 non-null    float64
 8   Director       999 non-null    object 
 9   Star1          999 non-null    object 
 10  Star2          999 non-null    object 
 11  Star3          999 non-null    object 
 12  Star4          999 non-null    object 
 13  No_of_Votes    999 non-null    int64  
 14  Gross          830 non-null    object 
dtypes: float64(2), int64(1), object(12)
memory usage: 117.2+ KB


* Nota-se a presença de dados nulo nas colunas `Certificate`, `Meta_score` e `Gross`.
* As colunas `Released_Year` e `Gross` devem ser alteradas o tipo de dado.

In [86]:
df.isnull().sum()

Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

In [87]:
df.describe()

,IMDB_Rating,Meta_score,No_of_Votes
count,999.000000,842.000000,9.990000e+02
mean,7.947948,77.969121,2.716214e+05
std,0.272290,12.383257,3.209126e+05
min,7.600000,28.000000,2.508800e+04
25%,7.700000,70.000000,5.547150e+04
50%,7.900000,79.000000,1.383560e+05
75%,8.100000,87.000000,3.731675e+05
max,9.200000,100.000000,2.303232e+06


In [88]:
df['Released_Year'].unique()

array(['1972', '2008', '1974', '1957', '2003', '1994', '1993', '2010',
       '1999', '2001', '1966', '2002', '1990', '1980', '1975', '2020',
       '2019', '2014', '1998', '1997', '1995', '1991', '1977', '1962',
       '1954', '1946', '2011', '2006', '2000', '1988', '1985', '1968',
       '1960', '1942', '1936', '1931', '2018', '2017', '2016', '2012',
       '2009', '2007', '1984', '1981', '1979', '1971', '1963', '1964',
       '1950', '1940', '2013', '2005', '2004', '1992', '1987', '1986',
       '1983', '1976', '1973', '1965', '1959', '1958', '1952', '1948',
       '1944', '1941', '1927', '1921', '2015', '1996', '1989', '1978',
       '1961', '1955', '1953', '1925', '1924', '1982', '1967', '1951',
       '1949', '1939', '1937', '1934', '1928', '1926', '1920', '1970',
       '1969', '1956', '1947', '1945', '1930', '1938', '1935', '1933',
       '1932', '1922', '1943', 'PG'], dtype=object)

In [89]:
df.query('Released_Year == "PG"')

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
965,Apollo 13,PG,U,140 min,"Adventure, Drama, History",7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,Tom Hanks,Bill Paxton,Kevin Bacon,Gary Sinise,269197,"173,837,933"


In [90]:
df.loc[df['Released_Year'] == 'PG', 'Released_Year'] = 1995

In [95]:
df['Released_Year'] = df['Released_Year'].astype(int)

In [91]:
df[df.duplicated(subset=['Series_Title'])]

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
135,Drishyam,2015,UA,163 min,"Crime, Drama, Mystery",8.2,Desperate measures are taken by a man who trie...,NaN,Nishikant Kamat,Ajay Devgn,Shriya Saran,Tabu,Rajat Kapoor,70367,"739,478"


In [92]:
df[df['Series_Title'] == 'Drishyam']

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
86,Drishyam,2013,U,160 min,"Crime, Drama, Thriller",8.3,A man goes to extreme lengths to save his fami...,NaN,Jeethu Joseph,Mohanlal,Meena,Asha Sharath,Ansiba,30722,NaN
135,Drishyam,2015,UA,163 min,"Crime, Drama, Mystery",8.2,Desperate measures are taken by a man who trie...,NaN,Nishikant Kamat,Ajay Devgn,Shriya Saran,Tabu,Rajat Kapoor,70367,"739,478"


In [96]:
set(df['Certificate'])

{'16',
 'A',
 'Approved',
 'G',
 'GP',
 'PG',
 'PG-13',
 'Passed',
 'R',
 'TV-14',
 'TV-MA',
 'TV-PG',
 'U',
 'U/A',
 'UA',
 'Unrated',
 nan}

In [126]:
genres = df['Genre']

In [139]:
def split_genres(col):
    genres = df[col].str.split(', ')

    return genres

In [141]:
df.apply(split_genres("Genre"))

In [142]:
genres[0].split(', ')

['Crime', 'Drama']

In [148]:
df_2 = df['Genre'].str.split(', ')

In [ ]:
pd.DataFrame(df_2).explode()